#### 네이버 뉴스 SUB 페이지 크롤링
    - 한개의 SUB페이지 크롤링 한 후 함수로 재구성
    - 02_1 번에서 수집된 데이터를 이용해서 여러 SUB 페이지 크롤링

In [26]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen


In [27]:
#뉴스 SUB페이지 문서 코드 받아오기
url = "https://news.naver.com/main/read.naver?mode=LSD&mid=sec&sid1=101&oid=001&aid=0012576808"

headers = {"User-Agent":
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

result = requests.get(url,headers=headers)

In [28]:
result

<Response [200]>

In [29]:
#result.text  전송된 소스코드를 문자열로 반환
#result.content 전송된소스코드를 row text 형식으로 반환 , 앞에 b코드가 붙어있음

In [30]:
bs_obj = BeautifulSoup(result.content,'html.parser')
#기사 제목 추출

title = bs_obj.find(id='articleTitle')
#title.text
date_time = bs_obj.find('span',{"class":'t11'})
#date_time.text



### 개발자 도구의 copy selector 사용 방법
- 크롤링 하려는 컨텐츠에서 검사 - 해당소스에서 단축메뉴 -copy - copy selector
    - 붙여넣기하면 선택자를 반환해주는데, 선택된 내용이 없는 경우가 있음
        - 잘못된 선택자를 생성했을 경우 -> 선택자 재구성
        - 동적으로 생성되는 태그인 경우에는 선택자가 만들어지지만 정적으로 크롤링은 불가능
        - 네이버 뉴스의 서브페이지 좋아요 수와 기사 댓글 수는 동적 생성해서 정적으로 크롤링이 불가능함

bs_obj.select("#main_content > div.article_header > div.article_info > div > div.article_btns > div.article_btns_left > div > a > span.u_likeit_text._count.num")

In [38]:
#sub 페이지에서 수집된 데이터 반환하는 함수 작성(서브 페이지 url을 파라미터로 전달)

def get_news_sub_info(url) :
    #1. 전달된 url에 접속 후 응답객체 반환
    
    headers = {"User-Agent":
              "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/92.0.4515.131 Safari/537.36"}

    result = requests.get(url, headers=headers)
    
    #2.bs4 객체 생성
    bs_obj=BeautifulSoup(result.content, "html.parser")
    
    #3.필요 데이터 추출
    # 기사 제목 추출(태그 객체)
    title = bs_obj.find(id='articleTitle')
    #기사 작성 시간 추출(태그 객체)
    date_time = bs_obj.find('span',{"class":'t11'})
    
    #4. 수집데이터 dict로 구성 후 반환
    dict_sub = {}
    dict_sub['기사제목'] = title.text
    dict_sub['기사 입력 시간'] = date_time.text
    
    return dict_sub

In [45]:
## main: 함수호출 코드
headers= {"User-Agent":
         "Mozilla/5.0(Window NT 10.0;Win64;x64) AppleWebKit/537.36 (KHTML,likeGecko) Chrome/87.0.4280.88 Safari/537.36"}

url = "https://news.naver.com/"
res = requests.get(url,headers=headers)
html = res.text
bs_obj = BeautifulSoup(html,"html.parser")

# 헤드라인 뉴스의 구성 ul, class : hdline_article_list
news_list = bs_obj.find("ul", {"class": "hdline_article_list"})
# news_list

# 모든 li 추출
lis = news_list.findAll("li")


link_list=[]
for li in lis :
    a_tag = li.find("a")
    link_list.append("https://news.naver.com"+a_tag["href"])


#link_list
for url in (link_list):
    res = get_news_sub_info(url)
    print(res)
    
    

{'기사제목': '1725명 신규확진,  "비수도권 600명대 넘어가며 확산세"', '기사 입력 시간': '2021.08.04. 오전 9:50'}
{'기사제목': '박범계 “변협, 로톡 가입 변호사 징계 않길 바란다”', '기사 입력 시간': '2021.08.04. 오전 10:55'}
{'기사제목': '文 \'숨은 태양광 찾아라\' 지시에…산업부 "태양광, 피크 기여도 11.1%"', '기사 입력 시간': '2021.08.04. 오전 11:19'}
{'기사제목': '힘 실리는 8월 기준금리 인상... 국내외 금융기관 임박 전망', '기사 입력 시간': '2021.08.04. 오후 12:05'}
{'기사제목': '맥도날드, 폐기 대상 식자재 사용 논란에 "재발 방지 총력"', '기사 입력 시간': '2021.08.04. 오전 9:38'}


In [46]:
##연습 문제

##get_news_sub_info 함수의 반환값에 기사 내용까지 포함되게 코드 수정
## 반환되는 결과값을 df로 구성하시오.

